# Plotly Dash Dashboard

This dashboard allows interactive exploration of the AI Text Detector results. It has three tabs for:
1. **EDA** – Visualizing data distributions (class balance, article length, etc.).
2. **Evaluation** – Showing model performance metrics (confusion matrix, ROC curves).
3. **Inference** – Providing an interface to input new text and get predictions with explanations.

Run this app to launch the dashboard locally and interact with the model.


In [8]:
import dash
from dash import dcc, html
import base64, logging
from pathlib import Path

# ——— 1) Auto‑locate your diagrams/ directory ————————————————
def find_diagrams_dir(start: Path = Path.cwd(), marker: str = "diagrams") -> Path:
    for folder in (start, *start.parents):
        candidate = folder / marker
        if candidate.is_dir():
            return candidate
    raise FileNotFoundError(f"Couldn’t locate a '{marker}/' directory under {start!s}")

DIAGRAMS_DIR = find_diagrams_dir()

# ——— 2) Helper to read & encode image files as base64 URIs ————————
def encode_image(filename: str) -> str:
    img_path = (DIAGRAMS_DIR / filename).resolve()
    if not img_path.exists():
        raise FileNotFoundError(f"Couldn’t find image {img_path}")
    with open(img_path, "rb") as f:
        b64 = base64.b64encode(f.read()).decode("utf-8")
    return f"data:image/png;base64,{b64}"

# ——— 3) Load all of your diagrams by their exact filenames ————————
class_dist_img                  = encode_image("class_distribution.png")
class_weights_img               = encode_image("class_weights.png")
length_dist_img                 = encode_image("length_distribution.png")
split_pie_img                   = encode_image("split_proportions_pie.png")
readability_bar_img             = encode_image("readability_sentiment_bar.png")
readability_bar_annotated_img   = encode_image("readability_sentiment_bar_annotated.png")
readability_box_img             = encode_image("readability_sentiment_box.png")
readability_dist_img            = encode_image("readability_sentiment_dist.png")
readability_dist_colored_img    = encode_image("readability_sentiment_dist_colored.png")
conf_matrix_img                 = encode_image("baseline_confusion_matrix.png")
pr_curve_img                    = encode_image("baseline_pr_curves.png")
roc_curves_img                  = encode_image("baseline_roc_curves.png")

# ——— 4) Define app layout ————————————————————————————————
app = dash.Dash(__name__)
app.title = "AI Text Detector Dashboard"

app.layout = html.Div([
    html.H1(
        "AI‑Generated Text Detection Dashboard",
        style={"textAlign": "center", "marginBottom": "1em"}
    ),
    dcc.Tabs(id="tabs", value="tab-eda", children=[
        dcc.Tab(label="EDA",        value="tab-eda"),
        dcc.Tab(label="Evaluation", value="tab-eval"),
        dcc.Tab(label="Inference",  value="tab-inf"),
    ]),
    html.Div(id="tab-content")  # Container to dynamically render tab content
])


In [9]:
from dash.dependencies import Input, Output  # Callback inputs/outputs for interactivity

# Callback to render content based on selected tab
@app.callback(Output("tab-content", "children"), Input("tabs", "value"))
def render_tab_content(tab):
    """Return the appropriate layout content for the selected tab."""
    if tab == "tab-eda":
        # EDA tab: show class distribution and length distribution plots
        return html.Div([
            html.H3("Exploratory Data Analysis", style={"textAlign": "center", "marginTop": "1em"}),
            html.Img(src=class_dist_img, style={"width": "45%", "display": "inline-block", "padding": "1em"}),
            html.Img(src=length_dist_img, style={"width": "45%", "display": "inline-block", "padding": "1em"}),
            html.P(
                "The dataset is fairly balanced across classes. "
                "Human-written texts are generally longer than AI-generated or AI-paraphrased ones.",
                style={"textAlign": "center", "fontStyle": "italic", "marginTop": "0.5em"}
            )
        ])
    elif tab == "tab-eval":
        # Evaluation content to be added in the next commit
        return html.Div()
    elif tab == "tab-inf":
        # Inference content to be added later
        return html.Div()
    # Fallback (should not normally happen)
    return html.Div()


In [10]:
if __name__ == "__main__":
    app.run_server(debug=True, port=8050)
